In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import openai
from transformers import pipeline

In [1]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


NameError: name 'pd' is not defined

In [ ]:
zero_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral:

"{}"
"""

few_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral.
Ejemplo 1: \"Me encantó la película\" → positivo
Ejemplo 2: \"No me gustó para nada\" → negativo
Ejemplo 3: \"Está bien, sin más\" → neutral
Texto: "{}"
Respuesta:"""

cot_prompt = """Analiza el siguiente texto y razona si es positivo, negativo o neutral.
Texto: "{}"
Primero, explica por qué. Luego, indica si es positivo, negativo o neutral."""

prompts = {
    "zero-shot": zero_shot_prompt,
    "few-shot": few_shot_prompt,
    "chain-of-thought": cot_prompt
}

In [2]:

def predict_openai(opinion, prompt_template):
    prompt = prompt_template.format(opinion)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message["content"].strip().lower()

results = {}

for method, prompt_template in prompts.items():
    predictions = []
    for opinion in df['opinion']:
        pred = predict_openai(prompt_template, opinion)
        pred = pred.lower()
        if "positivo" in pred:
            predictions.append("positiva")
        elif "negativo" in pred:
            predictions.append("negativa")
        elif "neutral" in pred:
            predictions.append("neutral")
        else:
            predictions.append("desconocido")  # Fallback
    results[method] = predictions

for method, preds in results.items():
    print(f"\n=== {method.upper()} ===")
    print(classification_report(df['label'], preds, zero_division=0))


NameError: name 'prompts' is not defined